#**유통 판매량 예측 및 재고 최적화**
## **단계3 : 모델링 및 비즈니스 평가**

# **0.미션**

* 단계1 다양한 모델링 수행
    * LSTM, CNN 등을 이용해서 다양한 시계열 모델링 수행
    * 성능 상위 2~3개 모델 선정
* 단계2 데이터 파이프라인 구축
    * test 데이터(raw data)로 부터 예측에 사용할 수 있는 형태로 데이터를 만드는 과정을 하나의 함수로 엮기
* 단계3 비즈니스 평가
    * 단계1에서 선정된 모델에 대해 test셋으로 평균 재고액 평가 수행

* 대상 상품(핵심 상품)

| Product_ID|Product_Code|SubCategory|Category|LeadTime|Price|
|----|----|----|----|----|----|
|3|DB001|Beverage|Drink|2|8|
|12|GA001|Milk|Food|3|6|
|42|FM001|Agricultural products|Grocery|3|5|


# **1.환경설정**

* 세부 요구사항
    * 구글콜랩
        * 구글 드라이브 바로 밑에 project 폴더를 만들고,
        * 데이터 파일을 복사해 넣습니다.
    - 라이브러리 로딩
        * 기본적으로 필요한 라이브러리를 import 하도록 코드가 작성되어 있습니다.
        * 필요하다고 판단되는 라이브러리를 추가하세요.


### **(1) 경로 설정**

* 구글 드라이브 연결

In [348]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [349]:
path = '/content/drive/MyDrive/AIVLE_SCHOOL_5th/언어지능_딥러닝/2024.04.24_미니프로젝트6차_실습자료/'

### **(2) 라이브러리 설치 및 불러오기**

In [350]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import joblib

from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm

from keras.models import Sequential
from keras.backend import clear_session
from keras.layers import Dense, LSTM, Conv1D, Flatten
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

import warnings
warnings.filterwarnings("ignore")

In [351]:
# 학습곡선 함수
def dl_history_plot(history):
    plt.plot(history['loss'], label='train_err', marker = '.')
    plt.plot(history['val_loss'], label='val_err', marker = '.')

    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    plt.grid()
    plt.show()

In [352]:
# 예측 결과 시각화
def plot_model_result(y_train, y_val, pred) :

    y_train = pd.Series(y_train)
    y_val = pd.Series(y_val)
    y_val.index = range(len(y_train), len(y_train) + len(y_val))

    pred = pd.Series(pred.reshape(-1,), index = y_val.index)

    # 전체 시각화
    plt.figure(figsize = (20,12))
    plt.subplot(2,1,1)
    plt.plot(y_train, label = 'train')
    plt.plot(y_val, label = 'val')
    plt.plot(pred, label = 'pred')
    plt.legend()
    plt.grid()

    plt.subplot(2,1,2)
    plt.plot(y_val, label = 'val')
    plt.plot(pred, label = 'pred')
    plt.legend()
    plt.grid()

    plt.show()

### **(3) 데이터 불러오기**

* **세부 요구사항**
    - 상품별로 저장한 데이터를 불러오시오.

In [353]:
oil_price = pd.read_csv(path + 'train/oil_price_train.csv')
orders = pd.read_csv(path + 'train/orders_train.csv')
sales = pd.read_csv(path + 'train/sales_train.csv')
products = pd.read_csv(path + 'train/products.csv')
stores = pd.read_csv(path + 'train/stores.csv')
amazon = pd.read_csv(path + 'train/AMZN.csv')


sales['Date'] = pd.to_datetime(sales['Date'] )
oil_price['Date'] = pd.to_datetime(oil_price['Date'] )
orders['Date'] = pd.to_datetime(orders['Date'] )
amazon['Date'] = pd.to_datetime(amazon['Date'] )
amazon = amazon.loc[amazon['Date'].between('2014-01-01', '2017-02-28')]

In [354]:
df_orders = pd.merge(orders, stores, on='Store_ID', how='left')
df_sales = pd.merge(sales, products, on='Product_ID', how='left')
df_sales = pd.merge(df_sales, oil_price, on='Date', how='left')
df = pd.merge(df_sales, df_orders, on=['Date', 'Store_ID'], how='left')
df = pd.merge(df, amazon, on='Date', how='left')

In [355]:
# df['Month'] = df['Date'].dt.month
# df['Weekday'] = df['Date'].dt.day_name()
# df = pd.get_dummies(df, columns=['Month', 'Weekday'], drop_first=True, dtype=int)
# df.sort_values(by='Date', inplace=True)

In [356]:
df = df.loc[(df['Store_ID'] == 44) & (df['Product_ID'] == 3)]
df.isna().sum()

Date               0
Store_ID           0
Qty                0
Product_ID         0
Product_Code       0
SubCategory        0
Category           0
LeadTime           0
Price              0
WTI_Price        357
CustomerCount      6
City               6
State              6
Store_Type         6
Open             357
High             357
Low              357
Close            357
Adj Close        357
Volume           357
dtype: int64

In [357]:
drop_cols = ['Product_ID', 'Store_ID', 'Product_Code', 'SubCategory', 'Category', 'City', 'State', 'Store_Type']
beverage = df.drop(columns=drop_cols, axis=1)
beverage.reset_index(drop=True, inplace=True)
beverage.head()

,Date,Qty,LeadTime,Price,WTI_Price,CustomerCount,Open,High,Low,Close,Adj Close,Volume
0,2014-01-01,0.0,2,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014-01-02,9853.0,2,8,95.14,4422.0,19.940001,19.9680,19.701000,19.898500,19.898500,42756000.0
2,2014-01-03,8647.0,2,8,93.66,4167.0,19.914499,20.1355,19.811001,19.822001,19.822001,44204000.0
3,2014-01-04,15153.0,2,8,NaN,5341.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2014-01-05,15606.0,2,8,NaN,5123.0,NaN,NaN,NaN,NaN,NaN,NaN


In [358]:
beverage['Qty'] = beverage['Qty'].shift(-2)
beverage['Qty'] = beverage['Qty'].fillna(method='ffill')

beverage['WTI_Price'] = beverage['WTI_Price'].fillna(method='ffill')
beverage['WTI_Price'] = beverage['WTI_Price'].fillna(method='bfill')

beverage['Open'] = beverage['Open'].fillna(method='ffill')
beverage['Open'] = beverage['Open'].fillna(method='bfill')

beverage['High'] = beverage['High'].fillna(method='ffill')
beverage['High'] = beverage['High'].fillna(method='bfill')

beverage['Low'] = beverage['Low'].fillna(method='ffill')
beverage['Low'] = beverage['Low'].fillna(method='bfill')

beverage['Close'] = beverage['Close'].fillna(method='ffill')
beverage['Close'] = beverage['Close'].fillna(method='bfill')

beverage['Adj Close'] = beverage['Adj Close'].fillna(method='ffill')
beverage['Adj Close'] = beverage['Adj Close'].fillna(method='bfill')

beverage['Volume'] = beverage['Volume'].fillna(method='ffill')
beverage['Volume'] = beverage['Volume'].fillna(method='bfill')

beverage.sort_values(by='Date', inplace=True)
beverage.drop(columns=['LeadTime', 'Date'], inplace=True)
beverage['CustomerCount'] = beverage['CustomerCount'].fillna(0)

beverage.isna().sum()

Qty              0
Price            0
WTI_Price        0
CustomerCount    0
Open             0
High             0
Low              0
Close            0
Adj Close        0
Volume           0
dtype: int64

# **2.모델링**

* **세부 요구사항**
    * 3차원 데이터 구조 만들기
        * timestep 수는 적절한 간격으로 한가지 경우만 지정을 합니다.
    * LSTM, CNN 기반 초기 모델 생성
        * 성능 보다는 코드 틀을 작성하는데 집중합시다.
        * 노드 혹은 필터 수와 크기는 초기값으로 적절하게 지정해 봅시다.
    * 데이터 파이프라인 함수 작성하기
        * input : raw data
        * output : x_train, x_val, y_train, y_val
        * 원본 데이터에서 일부를 떼어 놓고, 파이프라인 함수를 테스트 해 봅시다.

In [359]:
def temporalize(x, y, timesteps):
    output_X = []
    output_y = []
    for i in range(len(x) - timesteps + 1):
        output_X.append(x.iloc[i:i+timesteps])
        output_y.append(y.iloc[i+timesteps-1])
    return np.array(output_X), np.array(output_y)

def flatten(X):
    flattened_X = np.empty((X.shape[0], X.shape[2]))
    for i in range(X.shape[0]):
        flattened_X[i] = X[i, (X.shape[1]-1), :]
    return flattened_X

def scale(X, scaler):
    for i in range(X.shape[0]):
        X[i, :, :] = scaler.transform(X[i, :, :])
    return X

## **(1) LSTM 모델링**

### **1) 상품 : 3 - Beverage**

* **데이터 준비**

In [360]:
from sklearn.model_selection import train_test_split

target = 'Qty'
x = beverage.drop(target, axis=1)
y = beverage[target]

# 3차원
timesteps = 15
x_s3, y_s3 = temporalize(x, y, timesteps)

val_size = 60
x_s3_train, y_s3_train = x_s3[:-val_size], y_s3[:-val_size]
x_s3_val = x_s3[-val_size:]
y_s3_val = y_s3[-val_size:]

# 2차원으로 변환해서 스케일러 생성
scaler = MinMaxScaler().fit(flatten(x_s3_train))

# 3차원 데이터에 스케일러 적용
x_s3_train_s = scale(x_s3_train, scaler)
x_s3_val_s = scale(x_s3_val, scaler)

# y에 대한 스케일링 (최적화를 위해)
scaler_y = MinMaxScaler()
y_s3_train_s = scaler_y.fit_transform(y_s3_train.reshape(-1, 1))
y_s3_val_s = scaler_y.transform(y_s3_val.reshape(-1, 1))

In [361]:
x_s3_train_s.shape, y_s3_train_s.shape

((1078, 15, 9), (1078, 1))

In [362]:
x_s3_val_s.shape, y_s3_val_s.shape

((60, 15, 9), (60, 1))

* **모델링**

In [363]:
n_features = x_s3_train_s.shape[2]

clear_session()
m_LSTM = Sequential([LSTM(32, input_shape = (timesteps, n_features)),
                    Dense(1)])

m_LSTM.compile(optimizer = Adam(learning_rate = 0.01), loss='mse')

# EarlyStopping 콜백 정의
es = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
hist = m_LSTM.fit(x_s3_train_s, y_s3_train_s, epochs=100, batch_size=32,
                  validation_data=(x_s3_val_s, y_s3_val_s),
                  callbacks=[es]).history

Epoch 1/100
34/34 [==============================] - 3s 24ms/step - loss: 0.0319 - val_loss: 0.0216
Epoch 2/100
34/34 [==============================] - 0s 7ms/step - loss: 0.0180 - val_loss: 0.0216
Epoch 3/100
34/34 [==============================] - 0s 7ms/step - loss: 0.0178 - val_loss: 0.0282
Epoch 4/100
34/34 [==============================] - 0s 7ms/step - loss: 0.0170 - val_loss: 0.0214
Epoch 5/100
34/34 [==============================] - 0s 7ms/step - loss: 0.0169 - val_loss: 0.0237
Epoch 6/100
34/34 [==============================] - 0s 7ms/step - loss: 0.0169 - val_loss: 0.0246
Epoch 7/100
34/34 [==============================] - 0s 6ms/step - loss: 0.0175 - val_loss: 0.0276
Epoch 8/100
34/34 [==============================] - 0s 7ms/step - loss: 0.0169 - val_loss: 0.0317
Epoch 9/100
34/34 [==============================] - 0s 7ms/step - loss: 0.0172 - val_loss: 0.0220
Epoch 10/100
34/34 [==============================] - 0s 7ms/step - loss: 0.0167 - val_loss: 0.0270
Epoch 11

### **2) 상품 : 12 - Milk**

* **모델링**

### **3) 상품 : 42 - Agricultural products**

* **모델링**

## **(2) CNN 모델링**

### **1) 상품 : 3 - Beverage**

* **데이터 준비**

In [364]:
n_features = x_s3_train_s.shape[2]

clear_session()

CNN = Sequential([Conv1D(32, 5, input_shape = (timesteps, n_features), activation='relu', padding = 'same'),
                    Flatten(),
                    Dense(1)])

CNN.compile(optimizer= Adam(learning_rate = 0.01) ,loss='mse')
es = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
hist = CNN.fit(x_s3_train_s, y_s3_train_s,
               epochs=100, batch_size=32,
               validation_data=(x_s3_val_s, y_s3_val_s),
               callbacks=[es]).history

Epoch 1/100
34/34 [==============================] - 1s 8ms/step - loss: 0.0470 - val_loss: 0.0458
Epoch 2/100
34/34 [==============================] - 0s 4ms/step - loss: 0.0188 - val_loss: 0.0218
Epoch 3/100
34/34 [==============================] - 0s 4ms/step - loss: 0.0158 - val_loss: 0.0189
Epoch 4/100
34/34 [==============================] - 0s 3ms/step - loss: 0.0135 - val_loss: 0.0165
Epoch 5/100
34/34 [==============================] - 0s 4ms/step - loss: 0.0123 - val_loss: 0.0197
Epoch 6/100
34/34 [==============================] - 0s 3ms/step - loss: 0.0120 - val_loss: 0.0160
Epoch 7/100
34/34 [==============================] - 0s 3ms/step - loss: 0.0110 - val_loss: 0.0161
Epoch 8/100
34/34 [==============================] - 0s 3ms/step - loss: 0.0113 - val_loss: 0.0162
Epoch 9/100
34/34 [==============================] - 0s 4ms/step - loss: 0.0110 - val_loss: 0.0163
Epoch 10/100
34/34 [==============================] - 0s 3ms/step - loss: 0.0111 - val_loss: 0.0166
Epoch 11/

* **모델링**

### **2) 상품 : 12 - Milk**

* **모델링**

### **3) 상품 : 42 - Agricultural products**

* **모델링**

# **3.데이터 파이프라인 생성**

* 세부 요구사항
    * 데이터 파이프라인 함수 작성하기
        * input : raw data
        * output : x_test, y_test
        * test set으로 파이프라인 함수를 테스트 해 봅시다.

## **(1) 파이프라인 함수 생성**

In [365]:
def temporalize(x, y, timesteps):
    output_X = []
    output_y = []
    for i in range(len(x) - timesteps + 1):
        output_X.append(x.iloc[i:i+timesteps])
        output_y.append(y.iloc[i+timesteps-1])
    return np.array(output_X), np.array(output_y)

def flatten(X):
    flattened_X = np.empty((X.shape[0], X.shape[2]))
    for i in range(X.shape[0]):
        flattened_X[i] = X[i, (X.shape[1]-1), :]
    return flattened_X

def scale(X, scaler):
    for i in range(X.shape[0]):
        X[i, :, :] = scaler.transform(X[i, :, :])
    return X


def data_pipeline(data, store_id, product_id, timesteps, scaler_x, scaler_y):
    df_1 = pd.merge(data['orders'], data['stores'], on='Store_ID', how='left')
    df_2 = pd.merge(data['sales'], data['products'], on='Product_ID', how='left')
    df_2 = pd.merge(df_2, data['oil_price'], on='Date', how='left')
    df = pd.merge(df_2, df_1, on=['Date', 'Store_ID'], how='left')
    df = pd.merge(df, data['amazon'], on='Date', how='left')

    # df['Month'] = df['Date'].dt.month
    # df['Weekday'] = df['Date'].dt.day_name()
    # df = pd.get_dummies(df, columns=['Month', 'Weekday'], drop_first=True, dtype=int)
    df.sort_values(by='Date', inplace=True)

    df = df.loc[(df['Store_ID'] == store_id) & (df['Product_ID'] == product_id)]

    # 데이터 전처리
    drop_cols = ['Product_ID', 'Store_ID', 'Product_Code', 'SubCategory', 'Category', 'City', 'State', 'Store_Type']
    ret_df = df.drop(columns=drop_cols, axis=1)
    ret_df.reset_index(drop=True, inplace=True)

    ret_df['Qty'] = ret_df['Qty'].shift(-2)
    ret_df['Qty'] = ret_df['Qty'].fillna(method='ffill')

    ret_df['WTI_Price'] = ret_df['WTI_Price'].fillna(method='ffill')
    ret_df['WTI_Price'] = ret_df['WTI_Price'].fillna(method='bfill')

    ret_df['Open'] = ret_df['Open'].fillna(method='ffill')
    ret_df['Open'] = ret_df['Open'].fillna(method='bfill')

    ret_df['High'] = ret_df['High'].fillna(method='ffill')
    ret_df['High'] = ret_df['High'].fillna(method='bfill')

    ret_df['Low'] = ret_df['Low'].fillna(method='ffill')
    ret_df['Low'] = ret_df['Low'].fillna(method='bfill')

    ret_df['Close'] = ret_df['Close'].fillna(method='ffill')
    ret_df['Close'] = ret_df['Close'].fillna(method='bfill')

    ret_df['Adj Close'] = ret_df['Adj Close'].fillna(method='ffill')
    ret_df['Adj Close'] = ret_df['Adj Close'].fillna(method='bfill')

    ret_df['Volume'] = ret_df['Volume'].fillna(method='ffill')
    ret_df['Volume'] = ret_df['Volume'].fillna(method='bfill')

    ret_df.sort_values(by='Date', inplace=True)
    ret_df.drop(columns=['LeadTime', 'Date'], inplace=True)
    ret_df['CustomerCount'] = ret_df['CustomerCount'].fillna(0)

    print(ret_df.isna().sum())
    # data split
    target = 'Qty'
    x_test = ret_df.drop(target, axis=1)
    y_test = ret_df[target]

    x_test_s3, y_test_s3 = temporalize(x_test, y_test, timesteps)
    # 3차원 데이터에 스케일러 적용
    x_test_s3_s = scale(x_test_s3, scaler)

    # y에 대한 스케일링 (최적화를 위해)
    y_test_s3_s = scaler_y.transform(y_test_s3.reshape(-1, 1))
    return x_test_s3_s, y_test_s3_s

## **(2) test 데이터로 확인**

* test용 데이터셋들 로딩

In [366]:
oil_price_test = pd.read_csv(path + 'test/oil_price_test.csv')
orders_test = pd.read_csv(path + 'test/orders_test.csv')
sales_test = pd.read_csv(path + 'test/sales_test.csv')
products = pd.read_csv(path + 'test/products.csv')
stores = pd.read_csv(path + 'test/stores.csv')
amazon_test= pd.read_csv(path + 'test/AMZN.csv')

sales_test['Date'] = pd.to_datetime(sales_test['Date'] )
oil_price_test['Date'] = pd.to_datetime(oil_price_test['Date'] )
orders_test['Date'] = pd.to_datetime(orders_test['Date'] )
amazon_test['Date'] = pd.to_datetime(orders_test['Date'])
amazon_test = amazon_test.loc[amazon_test['Date'].between('2017-03-01', '2017-03-31')]

In [367]:
amazon_test.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-03-01,19.940001,19.9680,19.701000,19.898500,19.898500,42756000
1,2017-03-01,19.914499,20.1355,19.811001,19.822001,19.822001,44204000
2,2017-03-01,19.792500,19.8500,19.421000,19.681499,19.681499,63412000
3,2017-03-01,19.752001,19.9235,19.714500,19.901501,19.901501,38320000
4,2017-03-01,19.923500,20.1500,19.802000,20.096001,20.096001,46330000


In [368]:
data = {
    'orders': orders_test,
    'stores': stores,
    'sales': sales_test,
    'products': products,
    'oil_price': oil_price_test,
    'amazon': amazon_test
}

In [369]:
x_test, y_test = data_pipeline(data, 44, 3, timesteps, scaler, scaler_y)

Qty              0
Price            0
WTI_Price        0
CustomerCount    0
Open             0
High             0
Low              0
Close            0
Adj Close        0
Volume           0
dtype: int64


* 예측해보기

In [394]:
result = {}

pred_lstm = m_LSTM.predict(x_test)
# pred_lstm = scaler_y.inverse_transform(pred_lstm)
# print(pred_lstm.shape)
# pred_lstm = pred_lstm.reshape(-1, )

result['LSTM'] = {
    'rmse': mean_squared_error(y_test, pred_lstm, squared=False),
    'mae' : mean_absolute_error(y_test, pred_lstm),
    'mape': mean_absolute_percentage_error(y_test, pred_lstm),
    'R2_Score': r2_score(y_test, pred_lstm)
}

47/47 [==============================] - 0s 2ms/step


In [395]:
pred_cnn = CNN.predict(x_test)
# pred_lstm = scaler_y.inverse_transform(pred_lstm)
# print(pred_lstm.shape)
# pred_lstm = pred_lstm.reshape(-1, )
result['CNN'] = {
    'rmse': mean_squared_error(y_test, pred_cnn, squared=False),
    'mae' : mean_absolute_error(y_test, pred_cnn),
    'mape': mean_absolute_percentage_error(y_test, pred_cnn),
    'R2_Score': r2_score(y_test, pred_cnn)
}

47/47 [==============================] - 0s 2ms/step


In [372]:
pred_cnn, y_test

(array([[0.29604036],
        [0.29666558],
        [0.26520246],
        ...,
        [0.5639542 ],
        [0.5759339 ],
        [0.606206  ]], dtype=float32),
 array([[0.39951993],
        [0.39951993],
        [0.39951993],
        ...,
        [0.40864912],
        [0.44709401],
        [0.44709401]]))

# **4.비즈니스 평가**

* 세부 요구사항
    * 기본 시뮬레이션 함수가 제공됩니다.
        * 필요하다면 해당 함수를 수정해서 사용할 수 있습니다.
    * 시뮬레이션 함수 사용하여 재고 평가
        * 실제값, 예측값, 안전재고 수량을 넣고 일평균 재고 금액을 확인합니다.
        * 기회손실수량은 0으로 맞추도록 안전재고 수량을 조절합니다.
        * 평균재고금액을 확인합니다.
    * 데이터는 test 셋을 이용합니다.
        * 예측 후 재고 평가 시, 스케일링된 y를 원래대로 돌려 놓아야 합니다.
            * y_real = y_sacled * (y_max - y_min) + y_min
            * pred_real = pred * (y_max - y_min) + y_min

## **(1) 재고 시뮬레이터**

In [373]:
def inv_simulator(y, pred, safe_stock, price) :

    # 시뮬레이션 df 틀 만들기
    temp = pd.DataFrame({'y':y.reshape(-1,), 'pred':pred.reshape(-1,).round()})

    temp['base_stock'] = 0
    temp['close_stock'] = 0
    temp['order'] = 0
    temp['receive'] = 0

    # 시뮬레이션

    for i in range(len(temp)-2):  # 발주량은 leadtime 후 판매 예측량에 기초하므로 계산을 위해 마지막 leadtime 만큼의 행 제외
        if i == 0 : #첫 행. 2일 전 데이터가 없으므로,
            temp.loc[ i ,'receive'] = temp.loc[ i ,'y']  # 입고량은 실판매량으로 계산
            temp.loc[ i ,'base_stock'] = temp.loc[ i ,'receive'] + safe_stock  # 기초재고는 실판매량 + 안전재고로 계산

        elif i == 1 : # 둘째 행, 2일 전 데이터가 없음.
            temp.loc[ i ,'receive'] = temp.loc[ i ,'y'] # 입고량은 실판매량으로 계산
            temp.loc[ i ,'base_stock'] = temp.loc[ i ,'receive'] + temp.loc[ i-1 ,'close_stock']
        else :      # 나머지 전체 행.
            temp.loc[ i ,'receive'] = temp.loc[ i-2 ,'order']    # 입고량 = 2일전 발주량
            temp.loc[ i ,'base_stock'] = temp.loc[ i ,'receive'] + temp.loc[ i-1 ,'close_stock']  # 기초재고 = 입고량 + 전날 기말재고

        # 기말재고 = 기초재고 - 판매량,  만약 0보다 작으면 0으로.
        stock = round(temp.loc[i, 'base_stock'] - temp.loc[i, 'y'])
        temp.loc[i, 'close_stock'] = np.where(stock> 0, stock, 0)

        # 발주량 = 2일후 판매예측량 + 안전재고 - 기말재고,  만약 주문량이 0보다 작으면 0
        order = temp.loc[i+2, 'pred'] + safe_stock - temp.loc[i, 'close_stock']
        temp.loc[i, 'order'] = np.where(order>0,order, 0)

    # 기회손실 = 만약 (기초재고 - 실판매량)이 0보다 작으면, 그만큼이 기회손실
    temp['lost'] = np.where((temp['base_stock'] - temp['y'])<0, (temp['base_stock'] - temp['y']),0).round()

    inventory = temp[:len(temp)-2]

    # 측정지표 계산
    DailyStock = ((inventory['base_stock'] + inventory['close_stock'])/2)
    DailyTurnover = (inventory['y'] + inventory['lost']) / DailyStock

    AvgDailyStock = round(DailyStock.mean(),3)
    AvgDailyStockAmt = AvgDailyStock * price
    turnover = round(DailyTurnover.mean(), 3)
    lost_sum = inventory['lost'].sum()


    print(f'일평균 재고량     : {AvgDailyStock}')
    print(f'일평균 재고 금액  : {AvgDailyStockAmt}')
    print(f'일평균 재고회전율 : {turnover}')
    print(f'기회손실 수량     : {lost_sum}')

    return inventory

## **(2) 예측 결과 시뮬레이션(재고금액 평가)**


In [374]:
df.head()

,Date,Store_ID,Qty,Product_ID,Product_Code,SubCategory,Category,LeadTime,Price,WTI_Price,CustomerCount,City,State,Store_Type,Open,High,Low,Close,Adj Close,Volume
645,2014-01-01,44,0.0,3,DB001,Beverage,Drink,2,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1456,2014-01-02,44,9853.0,3,DB001,Beverage,Drink,2,8,95.14,4422.0,Saint Paul,\tMinnesota,1.0,19.940001,19.9680,19.701000,19.898500,19.898500,42756000.0
2267,2014-01-03,44,8647.0,3,DB001,Beverage,Drink,2,8,93.66,4167.0,Saint Paul,\tMinnesota,1.0,19.914499,20.1355,19.811001,19.822001,19.822001,44204000.0
3078,2014-01-04,44,15153.0,3,DB001,Beverage,Drink,2,8,NaN,5341.0,Saint Paul,\tMinnesota,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3889,2014-01-05,44,15606.0,3,DB001,Beverage,Drink,2,8,NaN,5123.0,Saint Paul,\tMinnesota,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [396]:
y_test_real = scaler_y.inverse_transform(y_test)
pred_lstm_real = scaler_y.inverse_transform(pred_lstm)

result['LSTM_inverse'] = {
    'rmse': mean_squared_error(y_test_real, pred_lstm_real, squared=False),
    'mae' : mean_absolute_error(y_test_real, pred_lstm_real),
    'mape': mean_absolute_percentage_error(y_test_real, pred_lstm_real),
    'R2_Score': r2_score(y_test_real, pred_lstm_real)
}


일평균 재고량     : 105705.399
일평균 재고 금액  : 845643.192
일평균 재고회전율 : 0.111
기회손실 수량     : 0.0


,y,pred,base_stock,close_stock,order,receive,lost
0,10153.0,6015.0,110153,100000,6245,10153,0.0
1,10153.0,6343.0,110153,100000,5913,10153,0.0
2,10153.0,6245.0,106245,96092,9862,6245,0.0
3,10153.0,5913.0,102005,91852,14419,5913,0.0
4,10153.0,5954.0,101714,91561,14883,9862,0.0
...,...,...,...,...,...,...,...
1490,10385.0,15735.0,118248,107863,7784,20054,0.0
1491,10385.0,15647.0,125316,114931,659,17453,0.0
1492,10385.0,15647.0,122715,112330,3232,7784,0.0
1493,10385.0,15590.0,112989,102604,13335,659,0.0


In [400]:
y_test_real = scaler_y.inverse_transform(y_test)
pred_cnn_real = scaler_y.inverse_transform(pred_cnn)


result['CNN_inverse'] = {
    'rmse': mean_squared_error(y_test_real, pred_cnn_real, squared=False),
    'mae' : mean_absolute_error(y_test_real, pred_cnn_real),
    'mape': mean_absolute_percentage_error(y_test_real, pred_cnn_real),
    'R2_Score': r2_score(y_test_real, pred_cnn_real)
}


일평균 재고량     : 95563.937
일평균 재고 금액  : 764511.496
일평균 재고회전율 : 0.123
기회손실 수량     : 0.0


,y,pred,base_stock,close_stock,order,receive,lost
0,10153.0,7523.0,100153,90000,6740,10153,0.0
1,10153.0,7539.0,100153,90000,6657,10153,0.0
2,10153.0,6740.0,96740,86587,10298,6740,0.0
3,10153.0,6657.0,93244,83091,13779,6657,0.0
4,10153.0,6885.0,93389,83236,13691,10298,0.0
...,...,...,...,...,...,...,...
1490,10385.0,13629.0,113179,102794,1346,15924,0.0
1491,10385.0,14307.0,109846,99461,5099,7052,0.0
1492,10385.0,14140.0,100807,90422,13910,1346,0.0
1493,10385.0,14560.0,95521,85136,19500,5099,0.0


In [398]:
for key in result:
    print(key, end='\t\t')
    print(result[key])

LSTM		{'rmse': 0.2083713787600677, 'mae': 0.1678469179069886, 'mape': 0.3880601903361789, 'R2_Score': -0.5709834980429471}
CNN		{'rmse': 0.18979308360321862, 'mae': 0.15117273984733162, 'mape': 0.33932171079264417, 'R2_Score': -0.30333559464651394}
LSTM_inverse		{'rmse': 5295.341854019165, 'mae': 4265.493731603833, 'mape': 0.38806019113521784, 'R2_Score': -0.5709835013594904}
CNN_inverse		{'rmse': 4823.211632161079, 'mae': 3841.752842273088, 'mape': 0.3393217112213792, 'R2_Score': -0.3033355938642144}


In [405]:
print('LSTM')
inv_simulator(y_test_real, pred_lstm_real, 3480048, 8)

print('CNN')
inv_simulator(y_test_real, pred_cnn_real, 3272564, 8)

LSTM
일평균 재고량     : 3485753.399
일평균 재고 금액  : 27886027.192
일평균 재고회전율 : 0.003
기회손실 수량     : 0.0
CNN
일평균 재고량     : 3278127.937
일평균 재고 금액  : 26225023.496
일평균 재고회전율 : 0.004
기회손실 수량     : 0.0


,y,pred,base_stock,close_stock,order,receive,lost
0,10153.0,7523.0,3282717,3272564,6740,10153,0.0
1,10153.0,7539.0,3282717,3272564,6657,10153,0.0
2,10153.0,6740.0,3279304,3269151,10298,6740,0.0
3,10153.0,6657.0,3275808,3265655,13779,6657,0.0
4,10153.0,6885.0,3275953,3265800,13691,10298,0.0
...,...,...,...,...,...,...,...
1490,10385.0,13629.0,3295743,3285358,1346,15924,0.0
1491,10385.0,14307.0,3292410,3282025,5099,7052,0.0
1492,10385.0,14140.0,3283371,3272986,13910,1346,0.0
1493,10385.0,14560.0,3278085,3267700,19500,5099,0.0
